# Arbitrary parameters on spatially detailed cells

It is often the case that biophysical properties *vary spatially* across the spatial extent of a neuron.  NeuroML has a provision for modelling the variability of the base conductance( $\bar g$ ) of ion channel distributions, as a user-specified function of distance from the soma (measured along neurites).  Still, modellers may desire more types of variability, and on different parameters as well.
This article shows how Eden's [<𝙲𝚞𝚜𝚝𝚘𝚖𝚂𝚎𝚝𝚞𝚙>]( extension_customsetup.ipynb ) extension can be used to model any kind of variability, on any attribute of a mechanism present on [spatially-detailed cells]( intro_spatial.ipynb ).
<!-- LATER all of them to the last? -->

As seen on the [main article]( extension_customsetup.ipynb ) about this extension, the model attributes of neurons can be customised in any conceivable way through `set cell` [statements]( extension_customsetup.ipynb#set-cell-statement ). For point neurons, the *location list* is always `all`.  For spatially-detailed cell models, the list can be much more specific - and we'll see right now how to use it. 

<div class="alert alert-info">
Note
    
This feature is still in development.
</div>

First, let's get a sample cell to try variability on:

In [ ]:
nmldb_cell_name = 'NMLCL000001'; zip_file_name = f'{nmldb_cell_name}.zip'
# Download the zip file with the model
import urllib.request
# Because NMLDB is hoving some issues with HTTPS, don't demand HTTPS verification
import ssl; ssl._create_default_https_context = ssl._create_unverified_context
urllib.request.urlretrieve(f'http://neuroml-db.org/GetModelZip?modelID={nmldb_cell_name}&version=NeuroML', zip_file_name)
# and unpack it
from zipfile import ZipFile
with ZipFile(zip_file_name, 'r') as zipp: zipp.extractall(nmldb_cell_name+'/')
# Find the cell's .nml file
import os
cell_filenames = [ nmldb_cell_name+'/'+name for name in os.listdir(nmldb_cell_name) if name.endswith('.cell.nml') ]
assert len(cell_filenames) == 1, "Didn't find a lonely cell nml file!"; cell_filename = cell_filenames[0]

## Sampling the cell's morphology

Let's get some facts about the neuron on which we'll define variability:

In [ ]:
import eden_simulator
cells_info = eden_simulator.experimental.explain_cell(cell_filenames[0])
# one cell in the model file, get it
cell_info = list(cells_info.values())[0]
nComps = len(cell_info['comp_midpoint'])
comp_midpoint = cell_info['comp_midpoint']
# list(cell_info.keys())

As mentioned in a [previous chapter]( intro_spatial.ipynb ), spatially-modelled cells eventually have to be split into discrete *compartments*, within which every biophysical quantity is considered to have the same value.  Hence, spatially-varying quantities on a modelled cell may be assigned values that differ for only individual compartments.  
**Note**: This is the way that *any* distribution, deterministic or stochastic, of parameters can be faithfully applied on neurons; the particular values are entirely controlled by the modeller. 

Given the information from `explain_cell`, it is straightforward to write the `set cell` statement: Set the *location list* field as a comma-separated list of `seg.fractionAlong` [cell locators]( extension_paths.rst#lems-paths-for-cell-locations ), and add as desired for variability, `multi` (to `set` one cell) or `multi_location`, `multi_cell` or  `multi_multi` (for multiple cells in a go), and the `values` lines that follow.

<div class="alert alert-info">
Note

When overriding parameters over spans of the neuron, make sure to *not* miss any spots (especially for [<𝚅𝚊𝚛𝚒𝚊𝚋𝚕𝚎𝚁𝚎𝚚𝚞𝚒𝚛𝚎𝚖𝚎𝚗𝚝>]( extension_customsetup.ipynb#Setting-VariableRequirements )s!).  
A reliable way is using [explain_cell]( python_api.rst#eden_simulator.experimental.explain_cell ) as shown here to get `cell-info`the list of compartments and their `segment`+`fractionAlong` midpoints, and the list of compartments per segment group of interest as `cell_info['segment_groups'][groupName]`.
</div>

The following examples will show how to implement different frequently used distributions for biophysical parameters.  In all cases, variability within a cell boils down to a vector of values, one for each compartment.  Here's a helper routine to write a [CustomSetup]( extension_customsetup.ipynb#set-cell-statement ) file, that `set`s a `cell` with different values all over:

In [ ]:
def set_cell_vals(cell_info, popname, instances, attribute, vals, units):
    locs = ','.join([str(loc) for loc in eden_simulator.experimental.GetLemsLocatorsForCell(cell_info)])
    lines = f'set cell {popname} {instances} {locs} {attribute} multi {units}\n'\
        'values '+'\t'.join(map(str,vals))+'\n'
    return lines

In the following, we'll use these routines to visualise the different cases of variability over the neuron. We'll use [PyVista]( https://pyvista.org ) for 3-D graphics, and EDEN's helper [pyvista.plot_neuron]( python_api.rst#eden_simulator.display.spatial.pyvista.get_neuron_mesh ) to colour a mesh per compartment.

In [ ]:
def plot_neuron(cell_info, vals, valsname, cmap=None, bgcol=None):
    import pyvista as pv
    from eden_simulator.display.spatial.pyvista  import get_neuron_mesh
    mesh = get_neuron_mesh(cell_info)
    mesh[valsname] = vals
    
    pl = pv.Plotter();
    pl.add_mesh(mesh, clim = None,cmap=cmap)
    pl.background_color = "#fefefe"
    if bgcol: pl.background_color = bgcol
    
    pl.camera.position = (000, 150, +1050)
    pl.camera.focal_point = (0.2, 0.3, -5.3)
    pl.camera.up = (0.0, 1., 1.)
    pl.camera.zoom(.8)
    # pl.camera.position = (1000, 150, -50)
    # pl.camera.focal_point = (0.2, 150.3, -50.3)
    # pl.camera.up = (0.0, 1.0, 0.6)
    # pl.camera.zoom(.8)
    return pl, mesh
def show_plot(pl):
    # displays won't show except with this trick... https://github.com/pyvista/pyvista/pull/5168
    import types
    scene = pl.show(jupyter_backend="html", return_viewer=True)
    scene_wrapper = types.SimpleNamespace(scene=scene,
        _repr_html_=lambda : scene.value if hasattr(scene,'value') else '')
    return scene_wrapper

## Variability as distance from a plane

One may wish to model a quantity with a *lateral* distribution over the neuron; that is to say, a quantity whose value is increasing (or decreasing) towards a certain *direction*.  Let $\hat{\mathbf{n}}$ be the [*unit vector*]( https://en.wikipedia.org/wiki/Unit_vector ) pointing to that direction.

Then one can assume a signed *directional distance metric* from a reference point $\mathbf{p}_0$, for every point in space $\mathbf{x}$, as follows:
$$
d_{\hat{\mathbf{n}},\mathbf{p}_0}(\mathbf{x}) = (\mathbf{x} - \mathbf{p}_0) \boldsymbol{\cdot} \hat{\mathbf{n}}
$$

which can be converted to "quantity" units as desired.  Note that $d$ has the same value over every *plane* perpendicular to $\hat{\mathbf{n}}$.

Let's sample $d$ for the midpoint of each compartment and see the result.

In [ ]:
import numpy as np
from numpy.linalg import norm
def unit_vec(v): return v / norm(v, axis=-1) 
def dist_by_plane(point_0, dist_direction, sample_points):
    n = unit_vec(dist_direction)
    return (sample_points - point_0).dot(n)
# Try n = [1,0,0] also
vals = dist_by_plane([0,0,0], [0,1,0], comp_midpoint)
pl,_ = plot_neuron(cell_info, vals, 'Distance from plane (μm)','reds','azure')
Plot_Vert = pl; Show_Vert = show_plot(pl); Show_Vert

## Variability as distance from an axle

Another sort of spatial distribution is as a function of *transverse distance* from an axle (designated by a reference point $\mathbf{p}_0$ on the axle and a direction vector $\hat{\mathbf{n}}$).  The *transverse* distance vector is thus:
$$
{\mathbf{d}}_{\perp,\hat{\mathbf{n}},\mathbf{p}_0}(\mathbf{x}) = \mathbf{r} - (\mathbf{r} \boldsymbol{\cdot} \hat{\mathbf{n}}) \hat{\mathbf{n}} , ~ \mathbf{r} = \mathbf{x} - \mathbf{p}_0
$$

in which case, the distance is the same over each *tube* centered at the axle.

Let's distribute the *reversal potential* $E_{_{Na}}$ of the ion chanel distribution `Na_all` as a function of that, then:

In [ ]:
def dist_by_axle(p0, axle_direction, x):
    n = unit_vec(axle_direction)
    d = (x - p0)
    aligned =  d.dot(n)[:,None]*n[None,:]
    return norm(d - aligned, axis=-1)

vals = dist_by_axle([0,0,0], [0,1,0], comp_midpoint) * (10/100) + 40
pl,_ = plot_neuron(cell_info, vals, 'Reversal Potential (mV)','blues','gray')
with open('cell_axle.txt', 'w') as f:
    f.write(set_cell_vals(cell_info, 'pop', 'all', 'biophysicalProperties/membraneProperties/Na_all/erev', vals, 'mV'))
Plot_Axle = pl; Show_Axle = show_plot(pl); Show_Axle

## Variability as by distance from a point in space

A simpler type of distribution is *radial distance* from a reference point $\mathbf{p}_0$, that's simply ${\mathbf{d}} = \mathbf{x} - \mathbf{p}_0$.  In this case, the distance is the same over each *sphere* centered at $\mathbf{p}_0$.

Let's distribute the *base conductivity* $\bar{g}_{_{Na}}$ of `Na_all` as a function of euclidean distance from a reference point:

In [ ]:
def dist_by_point(p0, x):
    return norm(x - p0, axis=-1)

vals = dist_by_point([0,100,0], comp_midpoint) * (3/10) + 10
pl,_ = plot_neuron(cell_info, vals, 'Conductance Density (S/cm²)','Greens','gray')
with open('cell_dist.txt', 'w') as f:
    f.write(set_cell_vals(cell_info, 'pop', 'all', 'biophysicalProperties/membraneProperties/Na_all/condDensity', vals, 'S_per_cm2'))
Plot_Dist = pl; Show_Dist = show_plot(pl); Show_Dist

## Randomised parameters

Finally, a quantity may simply exhibit a slight variation which could follow any random distribution.  As mentioned in the [previous article]( extension_customsetup.ipynb ), it's often better to assume some variation across neuron and mechanism instances, though this will also require more memory to store the variability.

For example, the *steady-state concentration* of a [\<𝚍𝚎𝚌𝚊𝚢𝚒𝚗𝚐𝙿𝚘𝚘𝚕𝙲𝚘𝚗𝚌𝚎𝚗𝚝𝚛𝚊𝚝𝚒𝚘𝚗𝙼𝚘𝚍𝚎𝚕\>]( https://docs.neuroml.org/Userdocs/Schemas/Cells.html#decayingpoolconcentrationmodel ) used for a `<species>` [distribution]( intro_spatial.ipynb#Adding-concentration-models-near-the-membrane )  may vary following an uniform distribution:

In [ ]:
rng = np.random.default_rng(seed=43264326)
vals = rng.uniform(0,100,nComps)

pl,_ = plot_neuron(cell_info, vals, 'Steady-state [Ca²⁺] (nM)','RdPu','azure')
with open('cell_rand.txt', 'w') as f:
    f.write(set_cell_vals(cell_info, 'pop', 'all', 'biophysicalProperties/intracellularProperties/ca_conc_all/restingConc', vals, 'nM'))
Plot_Rand = pl; Show_Rand = show_plot(pl); Show_Rand

Beside the above suggestions, a distribution may depdend on multiple factors, always following the intent of the modeller (or undisputed biophysical findings).  By evaluating the values of quantities for each compartment, distributions can be implemented in the simulation with full precision.

<!-- ## Specifying variability across a `<SegmentGroup>` NEXT

somewhere: also across one group only!
Note that the type, ignoring that apply per element and not for the cell type itself. -->

## Aside for NEURON users

Readers familiar with the NEURON simulator may have assigned spatial variability for parameters over the cell, with [𝚂𝚞𝚋𝚜𝚎𝚝𝙳𝚘𝚖𝚊𝚒𝚗𝙸𝚝𝚎𝚛𝚊𝚝𝚘𝚛]( https://nrn.readthedocs.io/en/8.2.6/guide/cellbuilder3.html ) or in the general case through HOC code (or the equvalent Python):
```
forsec <section_name or all>{
	for(x){
		// x is fraction along section for the compartment, and area(x), ri(x), x,y,z3d(x) and others are properties of the compartment.
		// Users can then set RANGE properties however they like based on the above properties.
	}
}
```

As shown, EDEN provides a similar mode of setting variability, albeit in a 'flatter' way.  `explain_cell` provides a list of per compartments and associated quantities over the whole cell, thus *one* loop over the explicitly listed compartments equates to `forsec ... for(x) ...` in HOC.  Variability over a section is then done by iterating only over the appropriate subset of compartments, provided by `cell_info['segment_groups'][groupName]` for the section's named `<segmentGroup>`.

---
Now as the last thing, we'll fetch screenshots to use in the documentation's [example gallery]( https://eden-simulator.org/gallery.html ) and hard-copy version.

In [ ]:
for pl, name in zip(
    [Plot_Vert, Plot_Axle, Plot_Dist, Plot_Rand],
    ['vert', 'axle', 'dist', 'rand']
):
    # # Hide scalar bar
    # list(pl.scalar_bars.values())[0].VisibilityOff()
    # # Change render size
    # pl.window_size = [1200,1200]
    # Set camera
    # pl.camera.position = (1000, 150, -50)
    # pl.camera.focal_point = (0.2, 150.3, -50.3)
    # pl.camera.up = (0.0, -1.0, 0.6)
    # pl.camera.zoom(.8)
    pl._on_first_render_request(); pl.render()
    pl.screenshot(f'_static/example_spatial_customsetup_{name}.png', transparent_background=False, scale=1) # NB: scale is int only, just set to whatever

In [ ]:
frames = []
for pl in [Plot_Vert, Plot_Axle, Plot_Dist, Plot_Rand]:
    # Hide scalar bar
    list(pl.scalar_bars.values())[0].VisibilityOff()
    # Change render size
    pl.camera.zoom('tight')
    pl.window_size = [600,600]
    # Set camera
    pl.camera.position = (800, 150, -50)
    pl.camera.focal_point = (0.2, 150.3, -50.3)
    pl.camera.up = (0.0, 1.0, 0.6)
    pl.background_color = 'k'
    pl.camera.zoom(1.6)
    # for whatever reason, updating the camera after show() needs this to apply the change
    pl._on_first_render_request(); pl.render()
    frames.append(pl.screenshot())
x= frames
im = np.hstack(np.hstack([[x[0],x[1]],[x[2],x[3]]]))
im.shape
from PIL import Image
im = Image.fromarray(im)
def crop(x, ltrb):
    l,t,r,b = ltrb
    return x.crop((int(l*x.size[0]), int(t*x.size[1]), int((1-r)*x.size[0]), int((1-b)*x.size[1])))
display(im)
thumb = crop(im, (0,0,0.1,0))
thumb.thumbnail((240, 200))
display(thumb)
thumb.save('_static/thumb_example_spatial_customsetup.png')
# im = np.block([[],[]])

In [ ]:
import io
outbuf = io.BytesIO()
frame_data = [Image.fromarray(x) for x in frames]
frame_data[0].save(outbuf, format='gif', save_all=True, append_images=frame_data[1:], duration=100, loop=0)
print('GIF size:',len(outbuf.getvalue()))
with open('_static/example_spatial_customsetup.gif','wb') as f: f.write(outbuf.getvalue())

And minimise the widgets of the plots.

In [ ]:
for pl in [Show_Vert, Show_Axle, Show_Dist, Show_Rand]:
    pl.scene.value = ''
    print(len(pl.scene.value))